# Preprocessing

In [ ]:
pip install googletrans==4.0.0-rc1

In [ ]:
pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=b4c51b66f28424d6d312439d4bc7f2c57acaec9e9fdb15a6337cac77d53fcabf
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 11.3 MB/s eta 0:00:00


In [ ]:
pip install --upgrade nltk

In [ ]:
import pandas as pd
import numpy as np
#from googletrans import Translator
from concurrent.futures import ThreadPoolExecutor
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import nltk
import string
import re
from langdetect import detect, LangDetectException
import contractions
from google.colab import drive

<h2> Load Dataset

In [ ]:
# load google drive to see the files in google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_csv_path = '/content/drive/MyDrive/Progetto-Vascon/Dataset/train.csv'
test_csv_path = '/content/drive/MyDrive/Progetto-Vascon/Dataset/test.csv'
test_labels_csv_path = '/content/drive/MyDrive/Progetto-Vascon/Dataset/test_labels.csv'

In [ ]:
# load csv files
train_df = pd.read_csv(train_csv_path)
X_test = pd.read_csv(test_csv_path)
y_test = pd.read_csv(test_labels_csv_path)

In [ ]:
train_df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [ ]:
X_test

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.
...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."


In [ ]:
y_test

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,-1,-1,-1,-1,-1,-1
153160,fffd7a9a6eb32c16,-1,-1,-1,-1,-1,-1
153161,fffda9e8d6fafa9e,-1,-1,-1,-1,-1,-1
153162,fffe8f1340a79fc2,-1,-1,-1,-1,-1,-1


In [ ]:
labels = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']

In [ ]:
# fill missing values
train_df.fillna(0, inplace=True)
#train_df_filtered = np.where(np.all(train_df[labels] == [0,0,0,0,0,0], axis=1))[0]
#random_elements = np.random.choice(train_df_filtered, size=round(len(train_df_filtered)*0.2), replace=False)
#train_df = train_df.drop(random_elements)

In [ ]:
# merge the X_test and the y_test on common attribute = id
test_df = pd.merge(X_test, y_test, on='id', how='inner')
test_df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,":If you have a look back at the source, the in...",-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,I don't anonymously edit articles at all.,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu...",-1,-1,-1,-1,-1,-1
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...,-1,-1,-1,-1,-1,-1
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ...",-1,-1,-1,-1,-1,-1
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the...",-1,-1,-1,-1,-1,-1


In [ ]:
# fill missing values
test_df.fillna(0, inplace=True)
test_df_filtered = np.where(np.all(test_df[labels] == [0,0,0,0,0,0], axis=1))[0]
random_elements = np.random.choice(test_df_filtered, size=round(len(test_df_filtered)*0.2), replace=False)
test_df = test_df.drop(random_elements)
test_df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...",-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,":If you have a look back at the source, the in...",-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,I don't anonymously edit articles at all.,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu...",-1,-1,-1,-1,-1,-1
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...,-1,-1,-1,-1,-1,-1
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ...",-1,-1,-1,-1,-1,-1
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the...",-1,-1,-1,-1,-1,-1


In [ ]:
# remove invalid rows in test dataset
test_df = test_df.loc[~(test_df == -1).any(axis=1)]
test_df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0
7,000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0
11,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0
13,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0
16,000663aff0fffc80,this other one from 1897,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
153150,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,0,0,0,0,0
153151,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,0,0,0,0,0
153154,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,0,0,0,0,0
153155,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0,1,0,1,0


In [ ]:
test_df.loc[(test_df == 1).any(axis=1)]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
21,00091c35fa9d0465,"== Arabs are committing genocide in Iraq, but ...",1,0,0,0,0,0
48,0013fed3aeae76b7,DJ Robinson is gay as hell! he sucks his dick ...,1,0,1,0,1,1
59,0017d4d47894af05,":Fuck off, you anti-semitic cunt. |",1,0,1,0,1,0
76,001d739c97bc2ae4,How dare you vandalize that page about the HMS...,1,0,0,0,0,0
81,001eff4007dbb65b,"::No, he is an arrogant, self serving, immatur...",1,0,1,0,1,0
...,...,...,...,...,...,...,...,...
152908,ff91c3d8a3e34398,NIGEL IS A CRAZY IDIOT!!!,1,0,0,0,1,0
153049,ffd49b8defd069d0,""" \n ::Well, now don't I feel stupid.... · """,0,0,0,0,1,0
153078,ffdf6854b41d9102,==Fourth Baldrick possibly being cleverer than...,1,0,0,0,0,0
153119,ffebe90c8d5acaba,""" \n\n == IRAN == \n That’s right, Iran. It wa...",1,0,1,0,0,0


<h2> Clean Text Functions

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
def pre_cleaning(text):
    text = re.sub(r'\r|\n', ' ', text.lower())  # Replace newline and carriage return with space, and convert to lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)  # Remove links and mentions
    text = re.sub(r'[^\x00-\x7f]', '', text)  # Remove non-ASCII characters
    text = re.sub(r'[¡!¿?.,;:¡¿!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]', '', text)
    return text

def perform_stemming(sentence):
    ps = PorterStemmer()
    words = word_tokenize(sentence)
    stemmed_words = [ps.stem(word) for word in words if word.lower() not in stopwords.words('english')]
    return ' '.join(stemmed_words)

def perform_lemmatization(sentence):
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(sentence)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words if word.lower() not in stopwords.words('english')]
    return ' '.join(lemmatized_words)

def preprocess_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.lower()
    return text

# Remove punctuations, stopwords, links, mentions and new line characters
def strip_all_entities(text):
    # Define stop words for text cleaning
    stop_words = set(stopwords.words('english'))
    banned_list = string.punctuation
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

# Clean hashtags at the end of the sentence, and keep those in the middle of the sentence by removing just the # symbol
def clean_hashtags(text):
    # Remove hashtags at the end of the sentence
    new_text = re.sub(r'(\s+#[\w-]+)+\s*$', '', text).strip()

    # Remove the # symbol from hashtags in the middle of the sentence
    new_text = re.sub(r'#([\w-]+)', r'\1', new_text).strip()

    return new_text

# Filter special characters such as & and $ present in some words
def filter_chars(text):
    return ' '.join('' if ('$' in word) or ('&' in word) else word for word in text.split())

# Function to check if the text is in English, and return an empty string if it's not
def filter_non_english(text):
    try:
        lang = detect(text)
    except LangDetectException:
        lang = "unknown"
    return text if lang == "en" else ""

# Remove repeated punctuation
def remove_repeated_punctuation(text):
    return re.sub(r'[\?\.\!]+(?=[\?\.\!])', '', text)

def remove_url_shorteners(text):
    return re.sub(r'(?:http[s]?://)?(?:www\.)?(?:bit\.ly|goo\.gl|t\.co|tinyurl\.com|tr\.im|is\.gd|cli\.gs|u\.nu|url\.ie|tiny\.cc|alturl\.com|ow\.ly|bit\.do|adoro\.to)\S+', '', text)

def remove_multiple_spaces_between_words(test):
    return ' '.join(test.split())

def truncate_text(text, max_words=300):
    words = text.split()
    if len(words) > 300:
      truncated_words = words[:max_words]
      text = ' '.join(truncated_words)
    return text

def check_text(text):
    words = text.split()
    for word in words:
        if len(word) > 100:
            return ''
    return text

<h2> Clean Text First

In [ ]:
# Function to call all the cleaning functions in the correct order
def clean_text(text):
    text = filter_non_english(text)
    text = pre_cleaning(text)
    text = clean_hashtags(text)
    text = filter_chars(text)
    text = truncate_text(text)
    text = remove_url_shorteners(text)
    text = remove_repeated_punctuation(text)
    return text

<h3> Train Cleaning

In [ ]:
train_df['comment_text'] = [clean_text(text) for text in train_df['comment_text']]
train_df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,d'aww! he matches this background colour i'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"hey man, i'm really not trying to edit war. it...",0,0,0,0,0,0
3,0001b41b1c6bb37e,""" more i can't make any real suggestions on im...",0,0,0,0,0,0
4,0001d958c54c6e35,"you, sir, are my hero. any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::and for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,you should be ashamed of yourself that is a ho...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"spitzer umm, theres no actual article for pros...",0,0,0,0,0,0
159569,fff125370e4aaaf3,and it looks like it was actually you who put ...,0,0,0,0,0,0


In [ ]:
print(f"There are around {int(train_df['comment_text'].duplicated().sum())} duplicated tweets, we will remove them.")
train_df.drop_duplicates("comment_text", inplace=True)
train_df

There are around 370 duplicated tweets, we will remove them.


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,d'aww! he matches this background colour i'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"hey man, i'm really not trying to edit war. it...",0,0,0,0,0,0
3,0001b41b1c6bb37e,""" more i can't make any real suggestions on im...",0,0,0,0,0,0
4,0001d958c54c6e35,"you, sir, are my hero. any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::and for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,you should be ashamed of yourself that is a ho...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"spitzer umm, theres no actual article for pros...",0,0,0,0,0,0
159569,fff125370e4aaaf3,and it looks like it was actually you who put ...,0,0,0,0,0,0


In [ ]:
# Remove rows where 'column' has text of length 0
train_df = train_df[train_df['comment_text'].str.len() > 0]

<h3> Test Cleaning

In [ ]:
test_df['comment_text'] = [clean_text(text) for text in test_df['comment_text']]
test_df

<ipython-input-89-aa45caebf522>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['comment_text'] = [clean_text(text) for text in test_df['comment_text']]


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,thank you for understanding. i think very high...,0,0,0,0,0,0
7,000247e83dcc1211,:dear god this site is horrible.,0,0,0,0,0,0
11,0002f87b16116a7f,"""::: somebody will invariably try to add relig...",0,0,0,0,0,0
13,0003e1cccfd5a40a,""" it says it right there that it is a type. th...",0,0,0,0,0,0
16,000663aff0fffc80,this other one from 1897,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
153150,fff8f64043129fa2,":jerome, i see you never got around to this! i...",0,0,0,0,0,0
153151,fff9d70fe0722906,==lucky bastard== heh you are famous now. i ki...,0,0,0,0,0,0
153154,fffa8a11c4378854,==shame on you all!== you want to speak about ...,0,0,0,0,0,0
153155,fffac2a094c8e0e2,mel gibson is a nazi bitch who makes shitty mo...,1,0,1,0,1,0


In [ ]:
print(f"There are around {int(test_df['comment_text'].duplicated().sum())} duplicated tweets, we will remove them.")
test_df.drop_duplicates("comment_text", inplace=True)
test_df

There are around 3413 duplicated tweets, we will remove them.


<ipython-input-90-2873be61b809>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop_duplicates("comment_text", inplace=True)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,thank you for understanding. i think very high...,0,0,0,0,0,0
7,000247e83dcc1211,:dear god this site is horrible.,0,0,0,0,0,0
11,0002f87b16116a7f,"""::: somebody will invariably try to add relig...",0,0,0,0,0,0
13,0003e1cccfd5a40a,""" it says it right there that it is a type. th...",0,0,0,0,0,0
16,000663aff0fffc80,this other one from 1897,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
153150,fff8f64043129fa2,":jerome, i see you never got around to this! i...",0,0,0,0,0,0
153151,fff9d70fe0722906,==lucky bastard== heh you are famous now. i ki...,0,0,0,0,0,0
153154,fffa8a11c4378854,==shame on you all!== you want to speak about ...,0,0,0,0,0,0
153155,fffac2a094c8e0e2,mel gibson is a nazi bitch who makes shitty mo...,1,0,1,0,1,0


In [ ]:
# Remove rows where 'column' has text of length 0
test_df = test_df[test_df['comment_text'].str.len() > 0]

<h2> Data Augmentation of the Train Set

In [ ]:
def back_translate(text, language):
    translator = Translator()
    try:
        # Attempt to translate the text to the specified language
        translated_text = translator.translate(str(text), dest=language).text

        # Translate the translated text back to the original language
        back_translated_text = translator.translate(translated_text, dest='en').text

        return back_translated_text

    except Exception as e:
        print("An error occurred during translation:", e)
        # Return the original text in case of an error
        print(text)

# Function to perform back translation in parallel
def process_row(row):
    # Target language for translation (you can choose a language other than English)
    target_language = 'fr'

    desired_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    y = np.array(row[desired_columns])

    text = row['comment_text']

    if len(text) > 500:
        text = text[:500]

    back_translated_text = back_translate(text, target_language)
    if back_translated_text != text:
        augmented_row = {'comment_text': back_translated_text,
                      'toxic': row['toxic'],
                      'severe_toxic': row['severe_toxic'],
                      'obscene': row['obscene'],
                      'threat': row['threat'],
                      'insult': row['insult'],
                      'identity_hate': row['identity_hate']}
        return augmented_row
    return None

In [ ]:
desired_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
filtered_df = train_df[(train_df[desired_columns] == 1).any(axis=1)]
filtered_df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,cocksucker before you piss around on my work,1,1,1,0,1,0
12,0005c987bdfc9d4b,hey. what is it. @ | talk . what is it. an exc...,1,0,0,0,0,0
16,0007e25b2121310b,"bye! don't look, come or think of comming back...",1,0,0,0,0,0
42,001810bf8c45bf5f,you are gay or antisemmitian? archangel white ...,1,0,1,0,1,1
43,00190820581d90ce,"fuck your filthy mother in the ass, dry!",1,0,1,0,1,0
...,...,...,...,...,...,...,...,...
159494,fef4cf7ba0012866,""" our previous conversation you fucking shit e...",1,0,1,0,1,1
159514,ff39a2895fc3b40e,you are a mischievious pubic hair,1,0,0,0,1,0
159541,ffa33d3122b599d6,your absurd edits your absurd edits on great w...,1,0,1,0,1,0
159546,ffb47123b2d82762,""" hey listen don't you ever! delete my edits e...",1,0,0,0,1,0


In [ ]:
train_df_aug=pd.DataFrame()

for index, row in filtered_df.iterrows():
    augmented_row = process_row(row)
    if augmented_row is not None:
        print(f"index: {index} -> {augmented_row['comment_text']}")
        train_df_aug = pd.concat([train_df_aug, pd.DataFrame([augmented_row])], ignore_index=True)

Output streaming troncato alle ultime 5000 righe.
index: 109723 -> Reerere Kevin Forsyth hates blacks. Kevin Forsyth lives in the basement of his two mothers .kevin Forsyth is a lover of Jimmy Wales
index: 109736 -> @ Ok, King of the Wikipedia Nazi! But I was not going to add the Sony Pictures Animation model to it and I learned my lesson. I just wanted to delete this note there because it is not necessary and stupid as you.Illy in hell, freedom of expression killing Nazi and you fuck in hell!
index: 109744 -> Bitch is che guevara Aimant beans!
index: 109761 -> "All I know is that you seem to be an idiotic man. Then you bring this ridiculous expert from Coldplay in this sick game and it is an ultimate joke. Is it supposed to be the music expertWikipedia resident? I mean, what kind of crazy madman there is a talented molecule to sing a song on "" jello "" like a castrated schoolboy? Did you hear how they play the guitar? Chugga-chugga-chugga, it is an alternative non -talent trash of th

In [ ]:
train_df_aug

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,", it is because I am a very stupid and distrau...",1,0,1,0,1,1
1,Nm hey every chief hateful (mainly a bitchass ...,1,0,1,0,1,0
2,"Hello, you.firt, i aoin o ie or u and kee or o...",1,0,1,1,1,0
3,Giving the slut excuses me but I spent hours t...,1,0,1,0,1,0
4,What would you say to let the dishonorable epi...,1,0,1,0,0,0
...,...,...,...,...,...,...,...
13586,"""Our previous conversation, damn shit, eating ...",1,0,1,0,1,1
13587,You are a bit malicious pubic,1,0,0,0,1,0
13588,Your absurd modifications of your absurd chang...,1,0,1,0,1,0
13589,"""Hey listening, is it never! Deleting my chang...",1,0,0,0,1,0


 <h2> Clean Text

In [ ]:
# Function to call all the cleaning functions in the correct order
def clean_text(text):
    words = text.split()
    for word in words:
        if len(word) > 100:
            return ''
    text = strip_all_entities(text)
    text = clean_hashtags(text)
    text = filter_chars(text)
    text = preprocess_text(text)
    text = perform_stemming(text)
    text = perform_lemmatization(text)
    text = remove_url_shorteners(text)
    print(text)
    return text

<h3> Train Cleaning

In [ ]:
train_df['comment_text'] = [clean_text(text) for text in train_df['comment_text']]
train_df


Output streaming troncato alle ultime 5000 righe.
see talk page articl question well talk page thank
object rape loot desecr inflict civil popul invad troop properli mention necessarili word pfainuk text frustrat three day violent disord almost entir popul town left cite loyalti philip v bourbon claimant compli requir exodu almost gibraltarian san roqu nearbi area notabl relev gibraltarian histori thu must mention pfainuk text mention episod propos
one first graphic novel descript one pictur topic state one first graphic novel fals graphic novel reach back victorian era neither one first graphic novel even one first modern era graphic novel even one first super hero theme graphic novel think level late 70 graphic novel way first field graphic novel asid reltiv earli one though first marvel comic publish think best altern descript would one earlier marvel comic graphic novel least fairli accur
remov indo aryan origion jatsand also much written cultur like folk danc bhangratheir game kab

<ipython-input-24-aa0822860258>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['comment_text'] = [clean_text(text) for text in train_df['comment_text']]


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explan edit made usernam hardcor metallica fan...,0,0,0,0,0,0
1,000103f0d9cfb60f,daww match background colour im seemingli stuc...,0,0,0,0,0,0
2,000113f07ec002fd,hey man im realli tri edit war guy constantli ...,0,0,0,0,0,0
3,0001b41b1c6bb37e,cant make real suggest improv wonder section s...,0,0,0,0,0,0
4,0001d958c54c6e35,sir hero chanc rememb page,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,second time ask view complet contradict covera...,0,0,0,0,0,0
159567,ffea4adeee384e90,asham horribl thing put talk page 128611993,0,0,0,0,0,0
159568,ffee36eab5c267c9,spitzer umm actual articl prostitut ring crunc...,0,0,0,0,0,0
159569,fff125370e4aaaf3,look like actual put speedi first version dele...,0,0,0,0,0,0


In [ ]:
Y = train_df[labels]
Y

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
159566,0,0,0,0,0,0
159567,0,0,0,0,0,0
159568,0,0,0,0,0,0
159569,0,0,0,0,0,0


In [ ]:
print(f"There are around {int(train_df['comment_text'].duplicated().sum())} duplicated tweets, we will remove them.")
train_df.drop_duplicates("comment_text", inplace=True)
train_df

There are around 1084 duplicated tweets, we will remove them.


<ipython-input-26-87074eef8d03>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop_duplicates("comment_text", inplace=True)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explan edit made usernam hardcor metallica fan...,0,0,0,0,0,0
1,000103f0d9cfb60f,daww match background colour im seemingli stuc...,0,0,0,0,0,0
2,000113f07ec002fd,hey man im realli tri edit war guy constantli ...,0,0,0,0,0,0
3,0001b41b1c6bb37e,cant make real suggest improv wonder section s...,0,0,0,0,0,0
4,0001d958c54c6e35,sir hero chanc rememb page,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,second time ask view complet contradict covera...,0,0,0,0,0,0
159567,ffea4adeee384e90,asham horribl thing put talk page 128611993,0,0,0,0,0,0
159568,ffee36eab5c267c9,spitzer umm actual articl prostitut ring crunc...,0,0,0,0,0,0
159569,fff125370e4aaaf3,look like actual put speedi first version dele...,0,0,0,0,0,0


In [ ]:
# Remove rows where 'column' has text of length 0
train_df = train_df[train_df['comment_text'].str.len() > 0]

In [ ]:
# Save DataFrame to a CSV file
train_df.to_csv('/content/drive/MyDrive/Progetto-Vascon/Dataset/cleaned_train.csv', index=False)

<h3> Test Cleaning

In [ ]:
test_df['comment_text'] = [clean_text(text) for text in test_df['comment_text']]
test_df

Output streaming troncato alle ultime 5000 righe.
wasnt asshol check fuck ip go shoot mouth
sockpuppetri case alignleft ip address mention connect sockpuppetri case plea refer wikipediasockpuppet investigationspoliticalscholar evid plea make sure make familiar guid respond case edit evid page
11review submiss 5144156186 sure reject consid notabl citat reliabl reiter hyman archiv guin world record largest collect magazin verifi offici recognis certifi guin citat use draft citat exampl reput establish notabl ppa profession publish asscoci support project citat includ hypebeast see video also note mani industri notabl talk archiv suppli testimoni unsur draft reject
outsid u peopl use standard north american f would last long main articl mikel
english languag interfac page lyriklineorg whitelist link kruger ad
articl goddamn stupid
sent wikipedia email email
well dont keep track much block see per arbcom rule iasson reason assum theyr sockpuppet use sockpuppet revert vandal ban
bee gee did

<ipython-input-93-aa45caebf522>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['comment_text'] = [clean_text(text) for text in test_df['comment_text']]


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,thank understand think highli would revert wit...,0,0,0,0,0,0
7,000247e83dcc1211,dear god site horribl,0,0,0,0,0,0
11,0002f87b16116a7f,somebodi invari tri add religion realli mean w...,0,0,0,0,0,0
13,0003e1cccfd5a40a,say right type type institut need case three l...,0,0,0,0,0,0
16,000663aff0fffc80,one 1897,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
153150,fff8f64043129fa2,jerom see never got around im surpris look exa...,0,0,0,0,0,0
153151,fff9d70fe0722906,lucki bastard heh famou kida envi congrat,0,0,0,0,0,0
153154,fffa8a11c4378854,shame want speak gay romanian,0,0,0,0,0,0
153155,fffac2a094c8e0e2,mel gibson nazi bitch make shitti movi much bu...,1,0,1,0,1,0


In [ ]:
print(f"There are around {int(test_df['comment_text'].duplicated().sum())} duplicated tweets, we will remove them.")
test_df.drop_duplicates("comment_text", inplace=True)
test_df

There are around 283 duplicated tweets, we will remove them.


<ipython-input-94-2873be61b809>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop_duplicates("comment_text", inplace=True)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
5,0001ea8717f6de06,thank understand think highli would revert wit...,0,0,0,0,0,0
7,000247e83dcc1211,dear god site horribl,0,0,0,0,0,0
11,0002f87b16116a7f,somebodi invari tri add religion realli mean w...,0,0,0,0,0,0
13,0003e1cccfd5a40a,say right type type institut need case three l...,0,0,0,0,0,0
16,000663aff0fffc80,one 1897,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
153150,fff8f64043129fa2,jerom see never got around im surpris look exa...,0,0,0,0,0,0
153151,fff9d70fe0722906,lucki bastard heh famou kida envi congrat,0,0,0,0,0,0
153154,fffa8a11c4378854,shame want speak gay romanian,0,0,0,0,0,0
153155,fffac2a094c8e0e2,mel gibson nazi bitch make shitti movi much bu...,1,0,1,0,1,0


In [ ]:
# Remove rows where 'column' has text of length 0
test_df = test_df[test_df['comment_text'].str.len() > 0]

In [ ]:
# Save DataFrame to a CSV file
test_df.to_csv('/content/drive/MyDrive/Progetto-Vascon/Dataset/cleaned_test.csv', index=False)

<h3> Aumented Cleaning


In [ ]:
aug_csv_path = '/content/drive/MyDrive/Progetto-Vascon/Dataset/cleaned_augmented.csv'
train_df_aug = pd.read_csv(aug_csv_path)

In [ ]:
# Remove rows where 'column' has text of length 0
train_df_aug = train_df_aug[train_df_aug['comment_text'].str.len() > 0]

In [ ]:
def clean_text(text):
    words = text.split()
    for word in words:
        if len(word) > 100:
            return ''

    text = pre_cleaning(text)
    text = clean_hashtags(text)
    text = filter_chars(text)
    text = truncate_text(text)
    text = remove_url_shorteners(text)
    text = remove_repeated_punctuation(text)
    text = strip_all_entities(text)
    text = clean_hashtags(text)
    text = filter_chars(text)
    text = preprocess_text(text)
    text = perform_stemming(text)
    text = perform_lemmatization(text)
    text = remove_url_shorteners(text)
    print(text)
    return text

In [ ]:
train_df_aug['comment_text'] = [clean_text(text) for text in train_df_aug['comment_text']]
train_df_aug

Output streaming troncato alle ultime 5000 righe.
damn read
administr cancel cours notfuck abus twat wolfkeep talk
fagot quit fuck mii sht niqqa
fuck
support vandal peruvianllama naconkantari use cool man
explan nonsens plea plea check banner top page publish comment pa controversi subject realli seriou implement defens ofsuch ridicul posit laughabl
piss told leav leav alon hate stop turn thing even chanc place photo tri understand put therewhi dont give peopl chanc wast space first turn back go suck cock bed tonight
realli guy wiki wow fag wrote poem mean go articl must delet read advertis shitti book
kubigula destroy realli angri put speedi delet tag destroy work thank kubigula delet work start hate kubigula neil916 realli rudetoo stupiddirig aiden2009
hello dick war back
fals accus kim given inappropri inappropri inappropri label inform ad mann talk page havent ad anyth fals inclus reliabl sourc well known strongli biasedtoward agw omit data hypocrit strang publish privileg revok
ty

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,coq piss work,1,1,1,0,1,0
1,heywho speak exclus group taliban wpdemandez s...,1,0,0,0,0,0
2,goodby look come dont think come back launcher,1,0,0,0,0,0
3,,1,0,1,0,1,1
4,fuck dirti mother as dri,1,0,1,0,1,0
...,...,...,...,...,...,...,...
16111,previou convers damn shit eat liber make ladi ...,1,0,1,0,1,1
16112,bit malici pubic,1,0,0,0,1,0
16113,absurd modif absurd chang larg white shark tot...,1,0,1,0,1,0
16114,hey listen never delet chang never bore wwe 2k...,1,0,0,0,1,0


In [ ]:
print(f"There are around {int(train_df_aug['comment_text'].duplicated().sum())} duplicated tweets, we will remove them.")
train_df_aug.drop_duplicates("comment_text", inplace=True)
train_df_aug

There are around 169 duplicated tweets, we will remove them.


,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,coq piss work,1,1,1,0,1,0
1,heywho speak exclus group taliban wpdemandez s...,1,0,0,0,0,0
2,goodby look come dont think come back launcher,1,0,0,0,0,0
3,,1,0,1,0,1,1
4,fuck dirti mother as dri,1,0,1,0,1,0
...,...,...,...,...,...,...,...
16111,previou convers damn shit eat liber make ladi ...,1,0,1,0,1,1
16112,bit malici pubic,1,0,0,0,1,0
16113,absurd modif absurd chang larg white shark tot...,1,0,1,0,1,0
16114,hey listen never delet chang never bore wwe 2k...,1,0,0,0,1,0


In [ ]:
# Remove rows where 'column' has text of length 0
train_df_aug = train_df_aug[train_df_aug['comment_text'].str.len() > 0]

In [ ]:
# Save DataFrame to a CSV file
train_df_aug.to_csv('/content/drive/MyDrive/Progetto-Vascon/Dataset/cleaned_augmented.csv', index=False)